In [20]:
import math
import matplotlib.pyplot as plt

In [21]:
class Element_Pricing_Model:
    # input: number of base tokens to buy/sell
    # output: number of fy tokens bought/sold with fee, number of fy tokens bought/sold without fee, fy token fee
    @staticmethod
    def calc_in_given_out(out,in_reserves,out_reserves,token_in,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = pow(k-pow(out_reserves-out,1-t),1/(1-t)) - in_reserves
        if token_in == "base":
            fee =  (out-without_fee)*g
            with_fee = without_fee+fee
            without_fee_or_slippage = pow(in_reserves/out_reserves,t)*out
        elif token_in == "fyt":
            fee =  (without_fee-out)*g
            with_fee = without_fee+fee
            without_fee_or_slippage = 1/pow(out_reserves/in_reserves,t)*out  
        return (without_fee_or_slippage,with_fee,without_fee,fee)
    
    # input: number of fy tokens to buy/sell
    # output: number of base tokens bought/sold with fee, number of base tokens bought/sold without fee, base token fee    
    @staticmethod
    def calc_out_given_in(in_,in_reserves,out_reserves,token_out,g,t):
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        without_fee = out_reserves - pow(k-pow(in_reserves+in_,1-t),1/(1-t))
        if token_out == "base":
            fee =  (in_-without_fee)*g
            with_fee = without_fee-fee
            without_fee_or_slippage = pow(in_reserves/out_reserves,t)*in_
        elif token_out == "fyt":
            fee =  (without_fee-in_)*g
            with_fee = without_fee-fee
            without_fee_or_slippage = 1/pow(out_reserves/in_reserves,t)*in_        
        return (without_fee_or_slippage,with_fee,without_fee,fee)

    
### Yield Space Method
# x^(1-t) + y^(1-t) = k
# -> y = (k - x^(1-t))^1/(1-t)

# (x-i)^(1-t) + (y+j)^(1-t) = k
# Sell j fyt for i base
# i = x - (k - (y+j)^(1-tg))^1/(1-tg)
# Buy i base for j fyt
# j = (k - (x-i)^(1-t/g))^1/(1-t/g) - y

class Yield_Pricing_Model:
    # input: number of base tokens to buy/sell
    # output: number of fy tokens bought/sold with fee, number of fy tokens bought/sold without fee, fy token fee
    @staticmethod
    def calc_in_given_out(out,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t/g) + pow(out_reserves,1-t/g)
        # calculate the number of fyts the specified amount of base tokens will cost with fees
        output_with_fee = pow(k-pow(out_reserves-out,1-t/g),1/(1-t/g)) - in_reserves
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        # calculate the number of fyts the specified amount of base tokens will cost without fees
        output_without_fee = pow(k-pow(out_reserves-out,1-t),1/(1-t)) - in_reserves
        # output_with_fee is greater than output_without_fee bc the fee adds additional cost
        fee =  output_with_fee - output_without_fee
        if in_reserves > out_reserves:
            without_fee_or_slippage = pow(in_reserves/out_reserves,t)*out
        else:
            without_fee_or_slippage = 1/pow(out_reserves/in_reserves,t)*out
        return (without_fee_or_slippage,output_with_fee,output_without_fee,fee)

    # input: number of fy tokens to buy/sell
    # output: number of base tokens bought/sold with fee, number of base tokens bought/sold without fee, base token fee    
    @staticmethod
    def calc_out_given_in(in_,in_reserves,out_reserves,g,t):
        k=pow(in_reserves,1-t*g) + pow(out_reserves,1-t*g)
        # calculate the number of base tokens the specified amount of fy tokens will cost with fees
        output_with_fee = in_reserves - pow(k-pow(out_reserves+in_,1-t*g),1/(1-t*g))
        k=pow(in_reserves,1-t) + pow(out_reserves,1-t)
        # calculate the number of base tokens the specified amount of fy tokens will cost without fees
        output_without_fee = in_reserves - pow(k-pow(out_reserves+in_,1-t),1/(1-t))
        # output_with_fee is greater than output_without_fee bc the fee adds additional cost
        fee =  output_with_fee - output_without_fee
        if in_reserves > out_reserves:
            without_fee_or_slippage = pow(in_reserves/out_reserves,t)*in_
        else:
            without_fee_or_slippage = 1/pow(out_reserves/in_reserves,t)*in_
        return (without_fee_or_slippage,output_with_fee,output_without_fee,fee)
    

class Market: 
    def __init__(self,x,y,g,t,pricing_model): 
        self.x=x
        self.y=y
        self.total_supply = x + y
        self.g=g
        self.t=t
        self.pricing_model=pricing_model
        self.x_orders = 0
        self.y_orders = 0
        self.x_volume = 0
        self.y_volume = 0
        self.cum_y_slippage=0
        self.cum_x_slippage=0
        self.cum_y_fees=0
        self.cum_x_fees=0
        self.starting_fyt_price=self.fyt_price()
    
    def apy(self):
        return ((self.y+self.total_supply)/self.x - 1) * 100
    
    def fyt_price(self):
        return 1/pow((self.y+self.total_supply)/self.x,self.t)
    
    def tick(self,step_size):
        self.t -= step_size
        
    
    def swap(self, amount, direction, token_in, token_out):
        if direction == "in":
            if token_in == "fyt" and token_out == "base":
                (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.pricing_model.calc_in_given_out(amount,self.y+self.total_supply,self.x,token_in,self.g,self.t)
                self.x -= output_with_fee
                self.y += amount
                self.cum_x_slippage += abs(without_fee_or_slippage-output_without_fee)
                self.cum_x_fees += fee
                self.x_orders+=1
                self.x_volume+=output_with_fee
            elif token_in == "base" and token_out == "fyt":
                (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.pricing_model.calc_in_given_out(amount,self.x,self.y+self.total_supply,token_in,self.g,self.t)
                self.x += amount
                self.y -= output_with_fee
                self.cum_y_slippage += abs(without_fee_or_slippage-output_without_fee)
                self.cum_y_fees += fee
                self.y_orders+=1
                self.y_volume+=output_with_fee
        elif direction == "out":
            if token_in == "fyt" and token_out == "base":
                (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.pricing_model.calc_out_given_in(amount,self.y+self.total_supply,self.x,token_out,self.g,self.t)
                self.x -= output_with_fee
                self.y += amount
                self.cum_x_slippage += abs(without_fee_or_slippage-output_without_fee)
                self.cum_x_fees += fee
                self.x_orders+=1
                self.x_volume+=output_with_fee
            elif token_in == "base" and token_out == "fyt":
                (without_fee_or_slippage,output_with_fee,output_without_fee,fee) = self.pricing_model.calc_out_given_in(amount,self.x,self.y+self.total_supply,token_out,self.g,self.t)
                self.x += amount
                self.y -= output_with_fee
                self.cum_y_slippage += abs(without_fee_or_slippage-output_without_fee)
                self.cum_y_fees += fee
                self.y_orders+=1
                self.y_volume+=output_with_fee   
        return (without_fee_or_slippage,output_with_fee,output_without_fee,fee)


In [3]:
x = 50000.0
y = 5000.0
g=0.05
t=0.9
display(Element_Pricing_Model.calc_out_given_in(10,2*y+x,x,"base",g,.9))
display(Element_Pricing_Model.calc_out_given_in(10,x,2*y+x,"fyt",g,.9))
display(Element_Pricing_Model.calc_in_given_out(10,x,2*y+x,"base",g,.9))
display(Element_Pricing_Model.calc_in_given_out(10,2*y+x,x,"fyt",g,.9))

(11.783196534742952, 8.409645798856582, 8.485376951291983, 0.07573115243540088)

(8.486661467891869,
 11.692040340720633,
 11.781095095495402,
 0.08905475477477012)

(8.486661467891869, 8.563549053399038, 8.487946371998987, 0.07560268140005065)

(11.783196534742952, 11.874563662946821, 11.78529872661602, 0.089264936330801)

In [29]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000.0
y_start = 5000.0
g=.05
t=.9
max_order_size=500
step_size=.0005
m = Market(x_start,y_start,g,t,Element_Pricing_Model)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
trades = []
init = {
    "percent_fee": float("{:.18f}".format(g)),
    "initial_x_reserves": float("{:.18f}".format(m.x)),
    "initial_y_reserves": float("{:.18f}".format(m.y)),
    "initial_apy": float("{:.18f}".format(m.apy())),
    "initial_fyt_price": float("{:.18f}".format(m.fyt_price())),
    "max_order_size": float("{:.18f}".format(max_order_size)),
    "total_supply": float("{:.18f}".format(m.total_supply)),
}
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,max_order_size)
    #print("amount: " + str(amount) + " total supply: " + str(m.x+2*m.y) + " x reserves: " + str(m.x) + " y reserves: " + str(m.y))
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
        
    if np.random.uniform(0,1) < 0.5:
        direction="in"
    else:
        direction="out"
        
    trade_input = {
        "time": float("{:.18f}".format(m.t)),
        "x_reserves": float("{:.18f}".format(m.x)),
        "y_reserves": float("{:.18f}".format(m.y)),
        "token_in": token_in,
        "amount_in": float("{:.18f}".format(amount)),
        "token_out": token_out,
        "direction": direction
    }
    (without_fee_or_slippage,with_fee,without_fee,fee) = m.swap(amount,direction,token_in,token_out)
    trade_output = {
        "x_reserves": float("{:.18f}".format(m.x)),
        "y_reserves": float("{:.18f}".format(m.y)),
        "amount_out": float("{:.18f}".format(with_fee)),
        "fee": float("{:.18f}".format(fee)),
    }
    
    trades.append({
        "input": trade_input,
        "output": trade_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum slippage x: " + str(m.cum_x_slippage))
print("Cum slippage y: " + str(m.cum_y_slippage))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending Reserve Ratio: " + str(m.apy()/100))
print("Ending Slippage Ratio: " + str(m.cum_y_slippage/m.cum_x_slippage - 1))
print("Ending Fee Ratio: " + str(m.cum_y_fees/m.cum_x_fees - 1))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

run={
    "init":init,
    "trades":trades
}

Starting APY: 19.999999999999996
Starting FYT Price: 0.8486661467891868
Starting X Reserves: 50000.0
Starting Y Reserves: 5000.0
Ending X Reserves: 27642.263003446493
Ending Y Reserves: 10033.691076271452
Num x orders: 914
Cum x volume: 240302.75045917122
Num y orders: 886
Cum y volume: 227583.15325781252
Cum slippage x: 55088.66608387034
Cum slippage y: 54663.08322513796
Cum fees x: 2747.1216263095075
Cum fees y: 2550.6541084374517
Ending Reserve Ratio: 1.3526905546106311
Ending Slippage Ratio: -0.007725415933732105
Ending Fee Ratio: -0.07151758989862822
Ending FYT Price: 0.9999999999999627
Ending Time: 4.3635234314720606e-14


In [30]:
import json
from genson import SchemaBuilder

with open('testTrades.json', 'w') as fp:
    json.dump(run, fp, indent=1)
    
builder = SchemaBuilder()
builder.add_object(run)
run_schema=builder.to_schema()

with open('test_vectors_schema.json', 'w') as fp:
    json.dump(run_schema, fp, indent=1)

In [126]:
import numpy as np

runs={}
np.random.seed(1)

t_max=0.25
t_min=.00025
step_size=.00025
num_steps=int(t_max/step_size)
times = np.arange(t_min, t_max+step_size, step_size) 
np.random.shuffle(times)
max_apy = 200
max_x_reserves=1000000.0

g=.05



x_orders=0
x_volume=0
y_orders=0
y_volume=0
trades = []
init = {
    "percent_fee": float("{:.18f}".format(g)),
    "t_max": float("{:.18f}".format(t_max)),
    "t_min": float("{:.18f}".format(t_min)),
    "num_tests": float("{:.18f}".format(num_steps)),
    "max_apy": float("{:.18f}".format(max_apy)),
    "max_x_reserves": float("{:.18f}".format(max_x_reserves)),
}
for t in times:
    # determine APY
    apy = np.random.uniform(0,max_apy)
    # determine base reserves
    x_reserves = np.random.uniform(0,max_x_reserves)
    # use apy and x_reserves to calculate y_reserves
    #    apy = (2*y+x)/x - 1
    #    y = (x*apy)/200
    y_reserves=(x_reserves*apy)/200
    
    # determine order size (bounded)
    amount = np.random.uniform(0,min(x_reserves,y_reserves))
    m = Market(x_reserves,y_reserves,g,t,Element_Pricing_Model)
    #print("time = " + str(t) + " apy = " + str(apy) + " x = " + str(x_reserves) + " y = " + str(y_reserves) + " amount = " + str(amount))
    #print("price = " + str(1/pow((2*y_reserves+x_reserves)/x_reserves,t)))
    #print("apy = " + str((2*y_reserves+x_reserves)/x_reserves -1 ))
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
        
    if np.random.uniform(0,1) < 0.5:
        direction="in"
    else:
        direction="out"
        
    trade_input = {
        "time": float("{:.18f}".format(m.t)),
        "x_reserves": float("{:.18f}".format(m.x)),
        "y_reserves": float("{:.18f}".format(m.y)),
        "total_supply": float("{:.18f}".format(m.total_supply)),
        "token_in": token_in,
        "amount_in": float("{:.18f}".format(amount)),
        "token_out": token_out,
        "direction": direction
    }
    (without_fee_or_slippage,with_fee,without_fee,fee) = m.swap(amount,direction,token_in,token_out)
    trade_output = {
        "x_reserves": float("{:.18f}".format(m.x)),
        "y_reserves": float("{:.18f}".format(m.y)),
        "amount_out": float("{:.18f}".format(with_fee)),
        "fee": float("{:.18f}".format(fee)),
    }
    
    trades.append({
        "input": trade_input,
        "output": trade_output
    });

run={
    "init":init,
    "trades":trades
}

In [127]:
import json
from genson import SchemaBuilder

with open('testTrades.json', 'w') as fp:
    json.dump(run, fp, indent=1)
    
builder = SchemaBuilder()
builder.add_object(run)
run_schema=builder.to_schema()

with open('test_vectors_schema.json', 'w') as fp:
    json.dump(run_schema, fp, indent=1)

In [ ]:
import numpy as np

runs={}
np.random.seed(1)


x_start = 50000.0
y_start = 5000.0
g=.95
t=.9
max_order_size=10
step_size=.0005
m = Market(x_start,y_start,g,t,Yield_Pricing_Model)
print("Starting APY: " + str(m.apy()))
print("Starting FYT Price: " + str(m.fyt_price()))
print("Starting X Reserves: " + str(m.x))
print("Starting Y Reserves: " + str(m.y))
x_orders=0
x_volume=0
y_orders=0
y_volume=0
trades = []
init = {
    "percent_fee": float("{:.18f}".format(g)),
    "initial_x_reserves": float("{:.18f}".format(m.x)),
    "initial_y_reserves": float("{:.18f}".format(m.y)),
    "initial_apy": float("{:.18f}".format(m.apy())),
    "initial_fyt_price": float("{:.18f}".format(m.fyt_price())),
    "max_order_size": float("{:.18f}".format(max_order_size)),
    "total_supply": float("{:.18f}".format(m.total_supply)),
}
while m.t-step_size >= 0:
    # determine order size
    amount = np.random.uniform(0,max_order_size)
    # buy fyt or base
    if np.random.uniform(0,1) < 0.5:
        token_in = "base"
        token_out = "fyt"
    else:
        token_in = "fyt"
        token_out = "base"
    trade_input = {
        "time": float("{:.18f}".format(m.t)),
        "x_reserves": float("{:.18f}".format(m.x)),
        "y_reserves": float("{:.18f}".format(m.y)),
        "token_in": token_in,
        "amount_in": float("{:.18f}".format(amount)),
        "token_out": token_out,
    }
    (without_fee_or_slippage,with_fee,without_fee,fee) = m.swap(amount,token_in,token_out)
    trade_output = {
        "x_reserves": float("{:.18f}".format(m.x)),
        "y_reserves": float("{:.18f}".format(m.y)),
        "amount_out": float("{:.18f}".format(with_fee)),
        "fee": float("{:.18f}".format(fee)),
    }
    
    trades.append({
        "input": trade_input,
        "output": trade_output
    });
    m.tick(step_size)

print("Ending X Reserves: " + str(m.x))
print("Ending Y Reserves: " + str(m.y))
print("Num x orders: " + str(m.x_orders))
print("Cum x volume: " + str(m.x_volume))
print("Num y orders: " + str(m.y_orders))
print("Cum y volume: " + str(m.y_volume))
print("Cum slippage x: " + str(m.cum_x_slippage))
print("Cum slippage y: " + str(m.cum_y_slippage))
print("Cum fees x: " + str(m.cum_x_fees))
print("Cum fees y: " + str(m.cum_y_fees))
print("Ending Reserve Ratio: " + str(m.apy()/100))
print("Ending Slippage Ratio: " + str(m.cum_y_slippage/m.cum_x_slippage - 1))
print("Ending Fee Ratio: " + str(m.cum_y_fees/m.cum_x_fees - 1))
print("Ending FYT Price: " + str(m.fyt_price()))
print("Ending Time: " + str(m.t))

run={
    "init":init,
    "trades":trades
}